In [1]:
import json
import re
from copy import deepcopy

with open('data/ilsdboundaries.json', 'rb') as jsonfile:
    _geodata = json.load(jsonfile)

with open('data/rc17.txt', 'r') as txtfile:
    _schoolsdata = txtfile.read().splitlines()
    
# Helper method return district object by district id
def lookupDistrict(district_id, districts=_geodata['objects']['ilsdboundaries']['geometries']):
    return [district for district in districts if district['properties']['rcdt'].replace('-', '') == district_id]

In [2]:
# These are the school districts that we'll be editing in the json.
# Need to deepcopy because otherwise, modifying district_objects modifies _geodata.
district_objects = deepcopy(_geodata['objects']['ilsdboundaries']['geometries'])

In [3]:
# Creating holder list of schools
schools = []

# Each entry is formatted as a string with
# columns of equal character length.
# See RC17_layout for features and location
# with index starting at 1. Last character is
# always a semicolon.
for i, school in enumerate(_schoolsdata):
    school_dict = {}
    school_dict['school_id'] = school[0:15]
    school_dict['school_name'] = school[19:52].strip()
    school_dict['district_name'] = school[53:86].strip()
    school_dict['school_enrollment'] = int(school[226:231].strip().replace(',',''))
    school_dict['district_enrollment'] = int(school[324:331].strip().replace(',',''))
    schools.append(school_dict)

In [4]:
for i, school in enumerate(schools):
    # Look up school district by its district ID.
    # First 11 characters of school ID are district-specific.
    # Last 4 are school-specific.
    # Sometimes duplicates in this data; will cull later.
    districts = [district for district in district_objects if district['properties']['rcdt'].replace('-', '') == school['school_id'][0:11]]
    for district in districts:
        # Try adding school enrollment to existing district enrollment.
        # If enrollment not already a key for the district
        # object, create it with current school.
        try:
            district['properties']['enrollment'] += school['school_enrollment']
        except KeyError:
            district['properties']['enrollment'] = school['school_enrollment']
    print ("%s: %s" % (i, school['school_name']))

0: Seymour High School
1: Seymour Elementary School
2: Liberty High School
3: Liberty Elementary School
4: Central High School
5: Central Junior High School
6: Central Elementary Grade School
7: Central 3-4 Middle School
8: Unity High School
9: Unity Elementary School
10: Unity Middle School
11: Quincy Sr High School
12: Quincy Jr High School
13: Adams Elementary School
14: Dewey Elementary School
15: Ellington Elementary School
16: Berrian School
17: Madison School
18: Monroe Elementary School
19: Washington Elementary School
20: Baldwin Intermediate
21: Brown County High School
22: Brown County Middle School
23: Brown County Elementary School
24: Beardstown Jr/Sr High School
25: Gard Elementary School
26: Virginia Jr/ Sr High School
27: Virginia Elem School
28: A-C Central High School
29: A-C Central Middle School
30: A-C Central Elementary School
31: Franklin Jr/Sr High School
32: Alexander Elem School
33: Franklin East Grade School
34: Waverly Jr/Sr  High School
35: Waverly Element

490: Komarek Elem School
491: S E Gross  Middle School
492: Brook Park Elem School
493: L J Hauser Jr High School
494: A F Ames Elem School
495: Blythe Park Elem School
496: Central Elem School
497: Hollywood Elem School
498: Gwendolyn Brooks Middle School
499: Percy Julian Middle School
500: William Beye Elem School
501: William Hatch Elem School
502: Oliver W Holmes Elem School
503: Irving Elem School
504: Abraham Lincoln Elem School
505: Longfellow Elem School
506: Horace Mann Elem School
507: Whittier Elem School
508: Lincoln Middle School
509: Prairie Oak School
510: Karel Havlicek Elem School
511: Jefferson Elem School
512: Unity Jr High School
513: Daniel Burnham Elem School
514: Cicero East Elem School
515: Columbus  East Elem School
516: Drexel Elem School
517: Goodwin Elementary School
518: Abe Lincoln Elem School
519: McKinley Elem School
520: T Roosevelt Elem School
521: Sherlock Elem School
522: Woodrow Wilson Elem School
523: Woodbine Elem School
524: Warren Park School
5

1506: Parkside Elem Community Academy
1507: Pasteur Elem School
1508: Peck Elem School
1509: Peirce Elem Intl Studies School
1510: Penn Elem School
1511: Washington  H Elem School
1512: Pershing Elem Humanities Magnet
1513: Peterson Elem School
1514: Pickard Elem School
1515: Pirie Elem Fine Arts & Academic C
1516: Plamondon Elem School
1517: Portage Park Elem School
1518: Horizon Sci Acad - Southwest Chtr
1519: Prescott Elem School
1520: Prussing Elem School
1521: Pulaski Intl Sch of Chicago
1522: Pullman Elem School
1523: Ravenswood Elem School
1524: Ray Elem School
1525: Reavis Elem Math & Sci Spec Schl
1526: Great Lakes Academy Charter ES
1527: Reilly Elem School
1528: Reinberg Elem School
1529: Revere Elem School
1530: Rogers Elem School
1531: Ruggles Elem School
1532: Ryder Elem Math & Sci Spec School
1533: Foundations College Prep Charter
1534: Sauganash Elem School
1535: Sawyer Elem School
1536: Sayre Elem Language Academy
1537: Ashe Elem School
1538: Scammon Elem School
1539: 

2486: Sam Rotolo Middle Sch
2487: Alice Gustafson Elem School
2488: J B Nelson Elem School
2489: Louise White Elem School
2490: H C Storm Elem School
2491: Grace McWayne Elementary School
2492: Hoover Wood Elem School
2493: West Aurora High School
2494: Jefferson Middle School
2495: Washington Middle School
2496: Jewel Middle School
2497: Herget Middle School
2498: Freeman Elem School
2499: Goodwin Elem School
2500: Greenman Elem School
2501: Hall Elem School
2502: Hill Elem School
2503: McCleery Elem School
2504: Nicholson Elem School
2505: Schneider Elem School
2506: Smith Elem School
2507: Fearn Elementary School
2508: East High School
2509: C F Simmons Middle School
2510: K D Waldo Middle School
2511: Henry W Cowherd Middle School
2512: Olney C Allen Elem School
2513: C M Bardwell Elem School
2514: W S Beaupre Elem School
2515: L D Brady Elem School
2516: G N Dieterich Elem School
2517: John Gates Elem School
2518: Nicholas A Hermes Elem School
2519: C I Johnson Elem School
2520: R

In [11]:
# Drop duplicates
new_district_objects = []
for d in district_objects:
    if d not in new_district_objects:
        new_district_objects.append(d)

In [16]:
# Finally, replace district data in _geodata with updated version.
_geodata['objects']['ilsdboundaries']['geometries'] = deepcopy(new_district_objects)

In [19]:
with open('data/illinois_school_districts.json', 'w') as jsonfile:
    jsonfile.write(json.dumps(_geodata))